In [55]:
#export
import mido
import time
import random
from dataclasses import dataclass

In [2]:
outport = mido.open_output('mido_out', virtual=True)

In [12]:
outport.send(mido.Message('note_on', note=60))

In [13]:
outport.send(mido.Message('note_off', note=60))

In [26]:
#export
def random_genome(length):
    alphabet = 'UDFBS'
    return ''.join([random.sample(alphabet, k=1)[0] for _ in range(length)])

In [29]:
random_genome(10)

'SFBBBDBSFB'

In [56]:
#export
@dataclass
class Note:
    pitch: int
    begin: float
    length: float

In [57]:
def events_length(genome):
    numeric = lambda g: {'U':1, 'D':2, 'F':3, 'B':5, 'S':7}[g]

    output = []
    t_pos_t = 0
    t_pos_p = 60
    t_len = 1
    saw_S = False
    
    c = iter(genome)

    try:
        while True:
            l = next(c)
            if l == 'S':
                if saw_S:
                    # second S, undo stamp and resize turtle
                    del output[-1]
                    t_len *= numeric(next(c)) / numeric(next(c))
                else:
                    saw_S = True
                    output.append(Note(t_pos_p, t_pos_t, t_len))
                    continue

            elif l == 'U':
                t_pos_p += 1
            elif l == 'D':
                t_pos_p -= 1
            elif l == 'F':
                t_pos_t += t_len
            elif l == 'B':
                t_pos_t -= t_len
            else:
                raise ValueError(f'Unexpected letter in genome: {l}')

            saw_S = False

    except StopIteration:
        return output

In [59]:
events_length(random_genome(40))

[Note(pitch=59, begin=-1, length=1),
 Note(pitch=59, begin=2, length=1),
 Note(pitch=58, begin=4, length=1),
 Note(pitch=57, begin=3.0, length=1.3333333333333333)]

In [62]:
@dataclass
class NoteOn:
    pitch: int
    time: float
        
@dataclass
class NoteOff:
    pitch: int
    time: float

In [71]:
def events_on_off(events):
    output = []
    for e in events:
        output.append(NoteOn(e.pitch, e.begin))
        output.append(NoteOff(e.pitch, e.begin+e.length))
        
    return sorted(output, key=lambda e: e.time - (1e-9 if isinstance(e,NoteOff) else 0))

In [75]:
l = events_on_off(events_length(random_genome(40)))
l

[NoteOn(pitch=60, time=0),
 NoteOn(pitch=61, time=0),
 NoteOff(pitch=60, time=1),
 NoteOff(pitch=61, time=1),
 NoteOn(pitch=61, time=1),
 NoteOff(pitch=61, time=2),
 NoteOn(pitch=62, time=7),
 NoteOff(pitch=62, time=8),
 NoteOn(pitch=60, time=8),
 NoteOn(pitch=62, time=8),
 NoteOff(pitch=60, time=9),
 NoteOff(pitch=62, time=9),
 NoteOn(pitch=61, time=10),
 NoteOff(pitch=61, time=11)]

In [80]:
def genome2midi(genome):
    midifile = mido.MidiFile(type=0)
    track = mido.MidiTrack()
    midifile.tracks.append(track)
    
    ppq = midifile.ticks_per_beat

    events = iter(events_on_off(events_length(genome)))

    e = None
    try:
        n = next(events)
        while True:
            e = mido.Message('note_on' if isinstance(n, NoteOn) else 'note_off', note=n.pitch)
            e_start = n.time
            n = next(events)
            e.time = round((n.time - e_start) * ppq)
            track.append(e)
        
    except StopIteration:
        if e is not None:
            track.append(e)
            
    return midifile

In [82]:
song = genome2midi(random_genome(40))
song

MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=61, velocity=64, time=160),
    Message('note_on', channel=0, note=67, velocity=64, time=320),
    Message('note_off', channel=0, note=61, velocity=64, time=0),
    Message('note_on', channel=0, note=65, velocity=64, time=0),
    Message('note_on', channel=0, note=65, velocity=64, time=480),
    Message('note_off', channel=0, note=65, velocity=64, time=0),
    Message('note_off', channel=0, note=65, velocity=64, time=0),
    Message('note_off', channel=0, note=67, velocity=64, time=0),
    Message('note_on', channel=0, note=64, velocity=64, time=480),
    Message('note_off', channel=0, note=64, velocity=64, time=0)])
])

In [100]:
for m in song.play():
    outport.send(m)